In [ ]:
# Install requirements
!pip install -q fastapi==0.68.1
!pip install -q python-multipart==0.0.5
!pip install -q uvicorn==0.15.0
!pip install -q nest-asyncio
!pip install -q pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 KB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 35.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import os
import requests

from fastapi import FastAPI
from pydantic import BaseModel
from pyngrok import ngrok
import nest_asyncio
import uvicorn

import math
import pandas as pd
from geopy import Point
from geopy.distance import distance
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm_notebook



In [ ]:
CRIME_DATA = '/content/crime_data.csv'
MODEL_PATH = '/content/bert_model.pth'
POLICE_DATA = '/content/pol_data.csv'
POP_DATA = '/content/po_data.csv'
TRANSIT_DATA = ''

In [ ]:

def get_routes(start, dest):
  KEY = 'jyLBvqJxo4n0KceZiCYfku4ixSlXvasX'
  URL = f'https://www.mapquestapi.com/directions/v2/alternateroutes?key={KEY}'

  payload = {
  "locations": [start, dest],
  "maxRoutes": 6,
  "timeOverage": 100
  }
  response = requests.post(URL, json=payload)
  if response.status_code == 200:
    data = response.json()
  return data

def process_data(data): 
  node_dict = {}
  nodes = []
  for node in data['route']['legs'][0]['maneuvers']:
    nodes.append(node['startPoint'])
    nodes[-1]['dist'] = node['distance']
    nodes[-1]['narr'] = node['narrative']
  node_dict[0] = nodes
  num_alternates = len(data['route']['alternateRoutes'])
  for i in range(num_alternates):
    nodes = []
    r = data['route']['alternateRoutes'][i]['route']['legs'][0]
    num_manuvs = len(r['maneuvers'])
    for j in range(num_manuvs):
      nodes.append(r['maneuvers'][j]['startPoint'])
      nodes[-1]['dist'] = r['maneuvers'][j]['distance']
      nodes[-1]['narr'] = r['maneuvers'][j]['narrative']
    node_dict[i+1] = nodes
  return node_dict


def get_km_range(lat, lng, k):
  p0 = Point(lat, lng)
  p1 = distance(kilometers=k).destination(point=p0, bearing=0)
  p2 = distance(kilometers=k).destination(point=p0, bearing=90) 
  lat_range = p1.latitude - p0.latitude
  lng_range = p2.longitude - p0.longitude
  return lat_range, lng_range

def get_distance(xa, ya, xb, yb):
  return math.sqrt((xa-xb)**2 + (ya-yb)**2)


def get_severity(category):
  weights = {
      'Poor / No Street Lighting': 1,
      'Stalking': 2,
      'Ogling/Facial Expressions/Staring': 2,
      'Commenting': 3,
      'Catcalls/Whistles': 3,
      'Taking pictures': 4,
      'Indecent exposure': 5,
      'Sexual Invites': 5,
      'Chain Snatching / Robbery': 8,
      'Touching /Groping': 8,
      'Rape / Sexual Assault': 10,
  }
  return weights[category]
  

def get_risk_factor(routes):
  risk_scores = []
  df = pd.read_csv(CRIME_DATA)
  for idx in routes.keys():
    route_risk = 0
    for node in routes[idx]:
      lat, lng = node['lat'], node['lng']
      dist = node['dist']
      if dist==0: dist=1 
      lat_r, lng_r = get_km_range(lat, lng, 2)
      crimes = df.loc[
          (df['Latitude'].between(lat-lat_r, lat+lat_r)) &
          (df['Longitude'].between(lng-lng_r, lng+lng_r))]
      # route_risk += crimes.shape[0]
      node_risk = 0
      for id, crime in crimes.iterrows():
        node_risk += get_severity(crime['Category'])*get_distance(
            crime['Latitude'], crime['Longitude'], lat, lng)
      route_risk += (node_risk/dist)
               
    risk_scores.append(route_risk)
  return risk_scores


In [ ]:
app = FastAPI()

class RoutePlanner(BaseModel):
    start_loc: str
    end_loc: str

class CrimeReportData(BaseModel):
    location: str
    incident: str


@app.post("/routes")
async def alternate_routes(inp_data: RoutePlanner):
  data = get_routes(inp_data.start_loc, inp_data.end_loc)
  node_dict = process_data(data)
  scores = get_risk_factor(node_dict)
  max_score = max(scores)
  scores = [i / max_score for i in scores]
  return {"routes": node_dict, "risk_scores": scores}

@app.post("/add_data")
async def add_to_dataframe():
  geolocator = Nominatim(timeout=10, user_agent="my-app")
  geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
  coord = geocode(location)
  df = pd.read_csv(CRIME_DATA)
  model = torch.load(MODEL_PATH)
  with torch.no_grad():
    category = model(incident.unsqueeze(0), attention_mask.unsqueeze(0))
  df.append("Category":str(category), "Latitude":coord.latitude, "Longitude":coord.longitude,
            "Incident":str(incident), "Location":str(location))

In [ ]:
auth_token = "2O8XV6yH8IEkUvZ77UC2nZpK1OP_555Xi375PfZLjk8ks1yAS"
os.system(f"ngrok authtoken {auth_token}")

0

In [ ]:
# Create tunnel
public_url = ngrok.connect(8000, port='8000', bind_tls=True)
!ps aux | grep ngrok     # check if tunnel exists

root         637  0.0  0.1 725928 22160 ?        Sl   06:24   0:00 /usr/local/lib/python3.9/dist-packages/pyngrok/bin/ngrok start --none --log=stdout
root         658  0.0  0.0   6904  3172 ?        S    06:24   0:00 /bin/bash -c ps aux | grep ngrok     # check if tunnel exists
root         660  0.0  0.0   6444   716 ?        S    06:24   0:00 grep ngrok


In [ ]:
nest_asyncio.apply()  # allow for asyncio to work within the Jupyter notebook cell
print(public_url)     # public URL for the server via a tunnel
uvicorn.run(app)      # run the FastAPI app using uvicorn

INFO:     Started server process [281]
INFO:uvicorn.error:Started server process [281]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


NgrokTunnel: "https://01a7-35-245-198-222.ngrok-free.app" -> "http://localhost:8000"
INFO:     149.154.161.203:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     103.93.250.20:0 - "POST /routes HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/uvicorn/protocols/http/h11_impl.py", line 373, in run_asgi
    result = await app(self.scope, self.receive, self.send)
  File "/usr/local/lib/python3.9/dist-packages/uvicorn/middleware/proxy_headers.py", line 75, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.9/dist-packages/fastapi/applications.py", line 208, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.9/dist-packages/starlette/applications.py", line 112, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.9/dist-packages/starlette/middleware/errors.py", line 181, in __call__
    raise exc from None
  File "/usr/local/lib/python3.9/dist-packages/starlette/middleware/errors.py", line 159, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.9/dist-pa

INFO:     2401:4900:3e22:998c:c193:32a:663b:b9f4:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     103.93.250.20:0 - "POST /routes HTTP/1.1" 200 OK
INFO:     103.93.250.20:0 - "POST /routes HTTP/1.1" 200 OK
INFO:     103.93.250.20:0 - "POST /routes HTTP/1.1" 200 OK
INFO:     2402:8100:3876:9ed1:1:1:54b0:2783:0 - "POST /routes HTTP/1.1" 200 OK
INFO:     2402:8100:3876:9ed1:1:1:54b0:2783:0 - "POST /routes HTTP/1.1" 200 OK
INFO:     2402:8100:3845:8e74:1:4:a8fc:400e:0 - "POST /routes HTTP/1.1" 200 OK
INFO:     2402:8100:3845:8e74:1:4:a8fc:400e:0 - "POST /routes HTTP/1.1" 200 OK
INFO:     2402:8100:3845:8e74:1:4:a8fc:400e:0 - "POST /routes HTTP/1.1" 200 OK


In [ ]:
ngrok.disconnect(public_url=public_url)  # kill tunnel